In this time,I use the very famous VGG19 network to classify the diital number.
Author SUNNK.
Time 2017/7/25.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [87]:
learning_rate = 0.001
training_iters = 2000000
batch_size = 256
display_step = 100

n_input = 784 
n_classes= 10 
dropout = 0.75

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) 

In [88]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def fc(x,W,b):
    return tf.nn.relu(tf.nn.xw_plus_b(x,W,b))

def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1_1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1_2 = conv2d(conv1_1, weights['wc1_2'], biases['bc1'])
    pool1=  maxpool2d(conv1_2, k=2)

    conv2_1 = conv2d(pool1, weights['wc2'], biases['bc2'])
    conv2_2 = conv2d(conv2_1, weights['wc2_2'], biases['bc2'])
    pool2 = maxpool2d(conv2_2, k=2)
    
    conv3_1 = conv2d(pool2, weights['wc3'], biases['bc3'])
    conv3_2 = conv2d(conv3_1, weights['wc3_3'], biases['bc3'])
    conv3_3 = conv2d(conv3_2, weights['wc3_3'], biases['bc3'])
    conv3_4 = conv2d(conv3_3, weights['wc3_3'], biases['bc3'])
    pool3 = maxpool2d(conv3_4, k=1)
    
    conv4_1 = conv2d(pool3, weights['wc4'], biases['bc4'])
    conv4_2 = conv2d(conv4_1, weights['wc4_4'], biases['bc4'])
    conv4_3 = conv2d(conv4_2, weights['wc4_4'], biases['bc4'])
    conv4_4 = conv2d(conv4_3, weights['wc4_4'], biases['bc4'])
    pool4 = maxpool2d(conv4_4, k=1)
                                                   
    conv5_1 = conv2d(pool4, weights['wc5'], biases['bc5'])
    conv5_2 = conv2d(conv5_1, weights['wc5_5'], biases['bc5'])
    conv5_3 = conv2d(conv5_2, weights['wc5_5'], biases['bc5'])
    conv5_4 = conv2d(conv5_3, weights['wc5_5'], biases['bc5'])
    pool5 = maxpool2d(conv5_4, k=1)


    fc1 = tf.reshape(pool3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    fc1 = tf.nn.dropout(fc1, dropout)
                                    
    #fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd1'])
    fc2 = tf.nn.relu(fc2)

    fc2 = tf.nn.dropout(fc2, dropout)

    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

In [89]:
weights = { #关于卷积神经网络参数计算问题可以参考http://blog.csdn.net/dcxhun3/article/details/46878999
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc1_2': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wc2_2': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    'wc3':tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3_3':tf.Variable(tf.random_normal([3, 3, 128, 128])),
    'wc4':tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wc4_4':tf.Variable(tf.random_normal([3, 3, 256, 256])),
    'wc5':tf.Variable(tf.random_normal([3, 3, 256, 512])),
    'wc5_5':tf.Variable(tf.random_normal([3, 3, 512, 512])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*128, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bc4': tf.Variable(tf.random_normal([256])),
     'bc5': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

VGG19结构：
class VGG19(object):
    def __init__(self,x,keeppro,classNum):
        self.X=x
        self.KEEPPRO=keeppro
        self.CLASSNUM=classNum
        self.buildCNN()
    def buildCNN(self):
        conv1_1=convLayer(self.X,3,3,1,1,64,"conv1_1")
        conv1_2=convLayer(conv1_1,3,3,1,1,64,"conv1_2")
        pool1=maxPoolLayer(conv1_2,2,2,2,2,"pool1")
        
        conv2_1=convLayer(pool1,3,3,1,1,128,"conv2_1")
        conv2_2=convLayer(conv2_1,3,3,1,1,128,"conv2_2")
        pool2=maxPoolLayer(conv2_2,2,2,2,2,"pool2")
        
        conv3_1=convLayer(pool2,3,3,1,1,256,"conv3_1")
        conv3_2=convLayer(conv3_1,3,3,1,1,256,"conv3_2")
        conv3_3=convLayer(conv3_2,3,3,1,1,256,"conv3_3")
        conv3_4=convLayer(conv3_3,3,3,1,1,256,"conv3_4")
        pool3=maxPoolLayer(conv3_4,2,2,2,2,"pool3")
        
        conv4_1=convLayer(pool3,3,3,1,1,512,"conv4_1")
        conv4_2=convLayer(conv4_1,3,3,1,1,512,"conv4_2")
        conv4_3=convLayer(conv4_2,3,3,1,1,512,"conv4_3")
        conv4_4=convLayer(conv4_3,3,3,1,1,512,"conv4_4")
        pool4=maxPoolLayer(conv4_4,2,2,2,2,"pool4")
        
        conv5_1=convLayer(pool4,3,3,1,1,512,"conv5_1")
        conv5_2=convLayer(conv5_1,3,3,1,1,512,"conv5_2")
        conv5_3=convLayer(conv5_2,3,3,1,1,512,"conv5_3")
        conv5_4=convLayer(conv5_3,3,3,1,1,512,"conv5_4")
        pool5=maxPoolLayer(conv5_4,2,2,2,2,"pool5")
        
        fcIn=tf.reshape(pool5,[-1,7*7*512])
        fc6=fcLayer(fcIn,7*7*512,4096,True,"fc6")
        dropout1=dropout(fc6,self.KEEPPRO)
        
        fc7=fcLayer(dropout1,4096,4096,True,"fc7")
        dropout2=dropout(fc7,self.KEEPPRO)
        
        self.fc8=fcLayer(dropout2,4096,self.CLASSNUM,True,"fc8")
    

In [90]:
pred = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [91]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,keep_prob:0.75})
        if step % display_step == 0:
            
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("iter" + str(step*batch_size) + ", loss= " + \
                  "{:.6f}".format(loss) + ", acc= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("finished")

    print("acc", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

迭代次数： 25600, 损失= nan, 训练准确率= 0.10547


KeyboardInterrupt: 